In [ ]:
"""
This notebook is used to go through all the Wing Scan Tifs, and segment them into the individual slides
as well as use OCR to automatically extract the name of the slide
"""

In [1]:
import os 
import cv2
import shutil
import easyocr
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from copy import deepcopy
from glob import glob
from tqdm import tqdm
from skimage.feature import match_template
from scipy import ndimage
from skimage.io import imread
from scipy.signal import fftconvolve
from skimage.feature import peak_local_max


In [36]:
DEBUG = False

# Get List of Img File Paths
img_fps = glob('../WWBEE24_images/WingScans/*.tif')
img_fps = sorted(img_fps)
# For reading the slide numbers
reader = easyocr.Reader(['en'])

for img_fp in tqdm(img_fps):
    if DEBUG:
        img_fp = '../WWBEE24_images/WingScans/Hive06_Sheet_04.tif'

    sheet = img_fp.split('/')[-1].split('.')[0]
    
    img = cv2.imread(img_fp)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Scale image so it's faster to process
    scale_factor = 10
    scaled_img = img_gray[::scale_factor,::scale_factor] > 150

    if DEBUG:
        plt.figure()
        plt.imshow(scaled_img)
    
    structured_element = (np.ones((330,117))*150).astype('uint8')

    # convolve structured element with scaled image, to find light slides in images
    convolved = fftconvolve(scaled_img, structured_element, mode='same', axes=None)
    points = peak_local_max(convolved, min_distance=50, threshold_abs=34000, num_peaks=24)

    # manual correction
    if img_fp == '../WWBEE24_images/WingScans/Hive01_Sheet_03.tif':
        points = points[:-1]
        points = np.array(list(points) + [[1490, 412]])

    if DEBUG:
        plt.figure()
        plt.imshow(convolved)
        plt.savefig(f'../for_presentation/{sheet}_convolution.jpg')
        # visualize the peaks that we found
        for i in range(len(points)):
            convolved[points[i][0]-10:points[i][0]+10, points[i][1]-10:points[i][1]+10] = 0
        plt.figure()
        plt.imshow(convolved)
        plt.savefig(f'../for_presentation/{sheet}_convolution_with_peaks.jpg')

    
    # Crop and rotate Individual Slides
    slide_size = np.multiply(structured_element.shape, scale_factor)
    
    #img_c = deepcopy(img)
    for i in range(len(points)):
        # convert the position back to space in the original image
        p = deepcopy(points[i].astype('float'))
        p[0] = int(img.shape[0]*p[0]/convolved.shape[0])
        p[1] = int(img.shape[1]*p[1]/convolved.shape[1])
        p = p.astype('int')
        #img_c[p[0]-100:p[0]+100, p[1]-100:p[1]+100] = 0
        start_x, end_x = max(int(p[0] - slide_size[0] // 2),0), int(p[0] + slide_size[0] // 2)
        start_y, end_y = max(int(p[1] - slide_size[1] // 2),0), int(p[1] + slide_size[1] // 2)
        slide = np.rot90( img[start_x:end_x, start_y:end_y])

        texts = reader.readtext(cv2.blur(slide, (11,11)))
        number = '-'.join(sorted([x[1] for x in texts]))
        number = number.lower().replace('o','0')


        # remove ones with mostly white backgrounds
        
        
        
        if DEBUG:
            plt.figure()
            plt.imshow(slide)
            if (slide < 100).sum() > 100_000:
                plt.title('YES!' + number + str((slide < 100).sum()))
            else:
                plt.title('NO!!!!!!!!!!!!!!!!!!!!!!!!!'+number + str((slide < 100).sum()))

        if (number != '0') and ((slide < 100).sum() > 100_000):
            slide_fp = '../slides/1_slides/' + sheet + '_slide' + number + '.png'
        else:
            slide_fp = '../slides/1_empty/' + sheet + '_slide_empty' + str(i) + '.png'

        if not DEBUG:
            if os.path.exists(slide_fp):
                print(f'Image Already Exists at {slide_fp}!!')                
            else:
                cv2.imwrite(slide_fp, slide)

    if DEBUG:
        break

  0%|                                                                                                                                                                                | 0/112 [00:00<?, ?it/s]

81
50
1
87
95
94
92
70
57
17
75
12
58
71
5
56
32
48
34
80
35
4
55
'-45


  1%|█▌                                                                                                                                                                      | 1/112 [00:16<29:53, 16.16s/it]

ww17
ww37
ww33
ww16
ww4
41
ww26
ww19
103
ww43
ww30
ww1
ww3
ww36
115
ww35
ww41
ww13
ww20
107
105
22
40
121


  2%|███                                                                                                                                                                     | 2/112 [00:31<29:02, 15.84s/it]

...

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 110/112 [36:31<00:40, 20.27s/it]

31
47
39
37
27
42
29
46
25
45
38
35
48
26
34
28
43
40
44
41
33
32
36
30


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 111/112 [36:52<00:20, 20.29s/it]

66
65
64
59
70
60
55
54
63
53
52
67
56
72
49
69
58
51
61
50
57
71
--62
68


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [37:11<00:00, 19.93s/it]


In [3]:
# Manual renamings - some automatic captures did not work as intended
orig_to_new = {
    "Hive01_Sheet_01_slide'-45":"Hive01_Sheet_01_slide45",
    "Hive01_Sheet_02_slide121":"Hive01_Sheet_02_slide121-x",
    "Hive01_Sheet_03_slideww2]":"Hive01_Sheet_03_slideww21-x",
    "Hive01_Sheet_03_slidewwl0":"Hive01_Sheet_03_slideww10",
    "Hive02_Sheet_01_slide35-`":"Hive02_Sheet_01_slide35",
    "Hive02_Sheet_01_slide)-47":"Hive02_Sheet_01_slide47",
    "Hive02_Sheet_03_slide-ww11":"Hive02_Sheet_03_slideww11",
    "Hive02_Sheet_03_slide-ww34":"Hive02_Sheet_03_slideww34",
    "Hive03_Sheet_01_slide(`-53":"Hive03_Sheet_01_slide53",
    "Hive03_Sheet_01_slidewwl7":"Hive03_Sheet_01_slideww17",
    "Hive05_Sheet_01_slideww3g":"Hive05_Sheet_01_slideww39",
    "Hive06_Sheet_02_slide62-^":"Hive06_Sheet_02_slide62",
    "Hive06_Sheet_03_slide5-77":"Hive06_Sheet_03_slide77",
    "Hive06_Sheet_03_slide-80":"Hive06_Sheet_03_slide80",
    "Hive06_Sheet_05_slide0-127":"Hive06_Sheet_05_slide127-x",
    "Hive06_Sheet_05_slide0-135":"Hive06_Sheet_05_slide135-x",
    "Hive06_Sheet_05_slide148-6":"Hive06_Sheet_05_slide148",
    "Hive07_Sheet_01_slide-38":"Hive07_Sheet_01_slide38",
    "Hive07_Sheet_03_slide4-83":"Hive07_Sheet_03_slide83",
    "Hive07_Sheet_04_slide118-d-x":"Hive07_Sheet_04_slide118-x",
    "Hive07_Sheet_04_slide119-d-y":"Hive07_Sheet_04_slide119-x",
    "Hive07_Sheet_04_slided-ww37":"Hive07_Sheet_04_slideww37",
    "Hive07_Sheet_05_slide.-160":"Hive07_Sheet_05_slide160",
    "Hive07_Sheet_05_slide119-d-x":"Hive07_Sheet_05_slide119-x",
    "Hive07_Sheet_05_slide158-6":"Hive07_Sheet_05_slide158",
    "Hive07_Sheet_06_slide118-d-x":"Hive07_Sheet_06_slide118-x",
    "Hive07_Sheet_06_slide137-d":"Hive07_Sheet_06_slide137",
    "Hive08_Sheet_02_slide4(-60":"Hive08_Sheet_02_slide60",
    "Hive08_Sheet_02_slide67-`":"Hive08_Sheet_02_slide67",
    "Hive08_Sheet_02_slide-53":"Hive08_Sheet_02_slide53",
    "Hive08_Sheet_02_slide-72":"Hive08_Sheet_02_slide72",
    "Hive08_Sheet_04_slide.-33":"Hive08_Sheet_04_slide33",
    "Hive08_Sheet_04_slide.-141":"Hive08_Sheet_04_slide141",
    "Hive08_Sheet_05_slide1-162":"Hive08_Sheet_05_slide162",
    "Hive08_Sheet_05_slide-192":"Hive08_Sheet_05_slide192",
    "Hive09_Sheet_01_slide33-s`":"Hive09_Sheet_01_slide33",
    "Hive09_Sheet_02_slide-54":"Hive09_Sheet_02_slide54",
    "Hive09_Sheet_05_slide.-170":"Hive09_Sheet_05_slide170",
    "Hive09_Sheet_05_slide161-4":"Hive09_Sheet_05_slide161",
    "Hive10_Sheet_01_slide29-4 ) `":"Hive10_Sheet_01_slide29",
    "Hive10_Sheet_01_slide-10":"Hive10_Sheet_01_slide10",
    'Hive10_Sheet_02_slide"-52':"Hive10_Sheet_02_slide52",
    "Hive10_Sheet_03_slide.-95":"Hive10_Sheet_03_slide95",
    "Hive10_Sheet_03_slide-p-.-90":"Hive10_Sheet_03_slide90",
    "Hive10_Sheet_04_slide130-7":"Hive10_Sheet_04_slide130",
    "Hive10_Sheet_04_slide'(-123":"Hive10_Sheet_04_slide123",
    "Hive10_Sheet_04_slide--124":"Hive10_Sheet_04_slide124",
    "Hive11_Sheet_01_slide.-ww17":"Hive11_Sheet_01_slideww17",
    "Hive11_Sheet_01_slide-1":"Hive11_Sheet_01_slide1",
    "Hive11_Sheet_01_slidew25":"Hive11_Sheet_01_slideww25",
    "Hive12_Sheet_01_slide-22":"Hive12_Sheet_01_slide22",
    "Hive12_Sheet_02_slide--ww10":"Hive12_Sheet_02_slideww10",
    "Hive12_Sheet_02_slide^ (-ww18":"Hive12_Sheet_02_slideww18",
    "Hive13_Sheet_01_slide(-30":"Hive13_Sheet_01_slide30",
    "Hive13_Sheet_01_slide-26":"Hive13_Sheet_01_slide26",
    "Hive13_Sheet_02_slidei <-ww23":"Hive13_Sheet_02_slideww23",
    "Hive13_Sheet_02_slideww32-x":"Hive13_Sheet_02_slideww32",
    "Hive15_Sheet_01_slide-40":"Hive15_Sheet_01_slide40",
    "Hive15_Sheet_02_slide2-ww31":"Hive15_Sheet_02_slideww31",
    "Hive15_Sheet_02_slide6-90-hive40":"Hive40_Sheet_02_slide90",
    "Hive15_Sheet_02_slide83-hive40":"Hive40_Sheet_02_slide83",
    "Hive15_Sheet_02_slide-ww24":"Hive15_Sheet_02_slideww24",
    "Hive16_Sheet_01_slide10-`":"Hive16_Sheet_01_slide10",
    "Hive16_Sheet_01_slide(-28":"Hive16_Sheet_01_slide28",
    "Hive16_Sheet_01_slide-13":"Hive16_Sheet_01_slide13",
    "Hive16_Sheet_01_slide- '-29":"Hive16_Sheet_01_slide29",
    "Hive16_Sheet_02_slide42-ww":"Hive16_Sheet_02_slideww42",
    "Hive16_Sheet_02_slide44-ww":"Hive16_Sheet_02_slideww44",
    "Hive16_Sheet_02_slide-33":"Hive16_Sheet_02_slide33",
    "Hive17_Sheet_02_slide52-7":"Hive17_Sheet_02_slide52",
    "Hive17_Sheet_02_slide-51":"Hive17_Sheet_02_slide51",
    "Hive18_Sheet_01_slide1'-22":"Hive18_Sheet_01_slide22",
    "Hive18_Sheet_01_slide12-{":"Hive18_Sheet_01_slide12",
    "Hive19_Sheet_02_slide3-39":"Hive19_Sheet_02_slide39",
    "Hive19_Sheet_02_slide4-52":"Hive19_Sheet_02_slide52",
    "Hive19_Sheet_02_slide37-`":"Hive19_Sheet_02_slide37",
    "Hive19_Sheet_02_slide%4-51":"Hive19_Sheet_02_slide51",
    "Hive19_Sheet_02_slide-45":"Hive19_Sheet_02_slide45",
    "Hive19_Sheet_03_slide-67":"Hive19_Sheet_03_slide67",
    "Hive26_Sheet_03_slide2-60":"Hive26_Sheet_03_slide60",
    "Hive26_Sheet_03_slide72-`":"Hive26_Sheet_03_slide72",
    'Hive26_Sheet_03_slide"-65':"Hive26_Sheet_03_slide65",
    "Hive27_Sheet_01_slide27-:":"Hive27_Sheet_01_slide27",
    "Hive27_Sheet_01_slide .-14":"Hive27_Sheet_01_slide14",
    "Hive27_Sheet_02_slide31-7,":"Hive27_Sheet_02_slide31",
    "Hive29_Sheet_02_slide'-45":"Hive29_Sheet_02_slide45",
    "Hive29_Sheet_03_slide1-84":"Hive29_Sheet_03_slide84",
    "Hive32_Sheet_01_slide(-74":"Hive32_Sheet_01_slide74",
    "Hive36_Sheet_01_slide37-s`":"Hive36_Sheet_01_slide37",
    "Hive36_Sheet_02_slide'-53":"Hive36_Sheet_02_slide53",
    "Hive36_Sheet_03_slide104-:":"Hive36_Sheet_03_slide104",
    "Hive36_Sheet_03_slide'-81":"Hive36_Sheet_03_slide81",
    "Hive37_Sheet_01_slide(-31":"Hive37_Sheet_01_slide31",
    "Hive37_Sheet_01_slide-'-11":"Hive37_Sheet_01_slide11",
    "Hive37_Sheet_02_slide'-47":"Hive37_Sheet_02_slide47",
    "Hive37_Sheet_02_slide-53":"Hive37_Sheet_02_slide53",
    "Hive37_Sheet_02_slide- --56":"Hive37_Sheet_02_slide56",
    'Hive38_Sheet_01_slide"-22':"Hive38_Sheet_01_slide22",
    "Hive39_Sheet_01_slide21-7":"Hive39_Sheet_01_slide21",
    "Hive39_Sheet_01_slide-28":"Hive39_Sheet_01_slide28",
    "Hive39_Sheet_02_slide'-77":"Hive39_Sheet_02_slide77",
    "Hive39_Sheet_03_slide104-rerarmed-wing":"Hive39_Sheet_03_slide104",
    "Hive40_Sheet_01_slide26-hy":"Hive40_Sheet_01_slide26",
    "Hive40_Sheet_02_slide5-61":"Hive40_Sheet_02_slide61",
    "Hive40_Sheet_02_slide'-59":"Hive40_Sheet_02_slide59",
    'Hive40_Sheet_03_slide"-84':"Hive40_Sheet_03_slide84",
    "Hive40_Sheet_03_slide&rawd-87-ce-lfag":"Hive40_Sheet_03_slide87",
    'Hive40_Sheet_04_slide".-ww40':"Hive40_Sheet_04_slideww40",
    "HiveC1_Sheet_01_slide2-4`":"HiveC1_Sheet_01_slide2",
    "HiveC2_Sheet_01_slide-3":"HiveC2_Sheet_01_slide3",
    "HiveC2_Sheet_02_slide'-'-43":"HiveC2_Sheet_02_slide43",
    "HiveC4_Sheet_01_slide.-38":"HiveC4_Sheet_01_slide38",
    "HiveC4_Sheet_01_slide31-4":"HiveC4_Sheet_01_slide31",
    "HiveC4_Sheet_01_slide-36":"HiveC4_Sheet_01_slide36",
    'HiveC5_Sheet_01_slide"3-6':"HiveC5_Sheet_01_slide6",
    "HiveC5_Sheet_03_slide--62":"HiveC5_Sheet_03_slide62",
    "Hive40_Sheet_03_slide-79":"Hive40_Sheet_03_slide79",
    "Hive05_Sheet_01_slidewv15":"Hive05_Sheet_01_slidewv16",
    "Hive10_Sheet_04_slide-117":"Hive10_Sheet_04_slide117"
}
remove = ['Hive08_Sheet_05_slide179-dek0rmed wings', 
         'Hive29_Sheet_03_slide90-li-del0rmed-n9s',
         'Hive38_Sheet_01_slide33-del0rmed-wiags',
          'Hive38_Sheet_03_slide(sings-81-del0cmecl']

In [12]:
# Update slide names from above
for old, new in orig_to_new.items():
    fp = glob(f'../slides/1_slides/{old}.png')
    print(fp)
    assert len(fp) <= 1

    if len(fp) == 0:
        print(old, new)
        continue
    fp = fp[0]
    print(fp)
    
    new_fp = fp.replace(old, new)
    if not os.path.exists(new_fp):
        shutil.move(fp, new_fp)
old, new, fp, new_fp

[]
Hive01_Sheet_01_slide'-45 Hive01_Sheet_01_slide45
[]
Hive01_Sheet_02_slide121 Hive01_Sheet_02_slide121-x
[]
Hive01_Sheet_03_slideww2] Hive01_Sheet_03_slideww21-x
[]
Hive01_Sheet_03_slidewwl0 Hive01_Sheet_03_slideww10
[]
Hive02_Sheet_01_slide35-` Hive02_Sheet_01_slide35
[]
Hive02_Sheet_01_slide)-47 Hive02_Sheet_01_slide47
[]
Hive02_Sheet_03_slide-ww11 Hive02_Sheet_03_slideww11
[]
Hive02_Sheet_03_slide-ww34 Hive02_Sheet_03_slideww34
[]
Hive03_Sheet_01_slide(`-53 Hive03_Sheet_01_slide53
[]
Hive03_Sheet_01_slidewwl7 Hive03_Sheet_01_slideww17
[]
Hive05_Sheet_01_slideww3g Hive05_Sheet_01_slideww39
[]
Hive06_Sheet_02_slide62-^ Hive06_Sheet_02_slide62
[]
Hive06_Sheet_03_slide5-77 Hive06_Sheet_03_slide77
[]
Hive06_Sheet_03_slide-80 Hive06_Sheet_03_slide80
[]
Hive06_Sheet_05_slide0-127 Hive06_Sheet_05_slide127-x
[]
Hive06_Sheet_05_slide0-135 Hive06_Sheet_05_slide135-x
[]
Hive06_Sheet_05_slide148-6 Hive06_Sheet_05_slide148
[]
Hive07_Sheet_01_slide-38 Hive07_Sheet_01_slide38
[]
Hive07_Sheet_03_

('Hive01_Sheet_02_slide121-x-x',
 'Hive01_Sheet_02_slide121-x',
 '../slides/1_slides/Hive01_Sheet_02_slide121-x-x.png',
 '../slides/1_slides/Hive01_Sheet_02_slide121-x.png')

In [47]:
# remove unuseful slides
for rem in remove:
    fp = glob(f'../slides/1_slides/{rem}*')

    assert len(fp) <= 1

    if len(fp) == 0:
        print(rem)
        continue

    fp = fp[0]
    os.remove(fp)